# ASTON VILLA v. MANCHESTER CITY - 2024-12-21 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [2]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Aston Villa"
away_side = "Manchester City"
matchday = "2024-12-21" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Aston Villa v. Manchester City


### Calculate Projected Goals

In [3]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.625 1.125
1.5 1.375
1.61 1.4
1.0093167701863353 0.8540372670807453 1.0714285714285714 0.8035714285714286

 ================== projected_goals ==================== 

HOME Aston Villa projected goals: 1.387810559006211
AWAY Manchester City projected goals:  1.2053571428571428


### Poisson Distribution of Goal Probabilities

In [4]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Aston Villa v. Manchester City [[7.47827750e-02 1.03784325e-01 7.20164908e-02 3.33150821e-02
  1.15587557e-02 3.20827264e-03 7.42079108e-04 1.47123603e-04
  2.55224612e-05]
 [9.01399520e-02 1.25097177e-01 8.68055916e-02 4.01565722e-02
  1.39324287e-02 3.86711434e-03 8.94470353e-04 1.77336486e-04
  3.07636809e-05]
 [5.43254175e-02 7.53933880e-02 5.23158700e-02 2.42015056e-02
  8.39677625e-03 2.33062695e-03 5.39078114e-04 1.06876900e-04
  1.85406113e-05]
 [2.18271767e-02 3.02919862e-02 2.10197692e-02 9.72381921e-03
  3.37370474e-03 9.36412613e-04 2.16593885e-04 4.29416116e-05
  7.44935275e-06]
 [6.57738582e-03 9.12816550e-03 6.33408223e-03 2.93016873e-03
  1.01662978e-03 2.82177908e-04 6.52682467e-05 1.29399946e-05
  2.24478264e-06]
 [1.58561980e-03 2.20053990e-03 1.52696625e-03 7.06379963e-04
  2.45080393e-04 6.80250314e-05 1.57343095e-05 3.11946297e-06
  5.41152957e-07]
 [3.18539691e-04 4.42072747e-04 3.06756613e-04 1.41906689e-04
  4.92349003e-05 1.36657429e-05 3.16091038e-06 6.266778

### Under Market

In [5]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Aston Villa v. Manchester City
    Goals      Prob  Implied Odds
0    0.5  0.074783     13.372063
1    1.5  0.268707      3.721525
2    2.5  0.520146      1.922537
3    3.5  0.737487      1.355955


In [6]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Aston Villa v. Manchester City
    Goals      Prob  Implied Odds
0    0.5  0.925197      1.080851
1    1.5  0.731273      1.367479
2    2.5  0.479834      2.084056
3    3.5  0.262492      3.809635


### 1 x 2 Probability

In [7]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                 winner      prob  implied_odds
0      Aston Villa (1)  0.411215      2.431817
1             Draw (x)  0.263008      3.802172
2  Manchester City (2)  0.325757      3.069773


# Full Results Summary:

In [8]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Aston Villa v. Manchester City


        ~ ~ moneyline market ~ ~

                 winner      prob  implied_odds
0      Aston Villa (1)  0.411215      2.431817
1             Draw (x)  0.263008      3.802172
2  Manchester City (2)  0.325757      3.069773


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.925197      1.080851
1    1.5  0.731273      1.367479
2    2.5  0.479834      2.084056
3    3.5  0.262492      3.809635


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.074783     13.372063
1    1.5  0.268707      3.721525
2    2.5  0.520146      1.922537
3    3.5  0.737487      1.355955

 ~~ MyBookie Odds for Aston Villa v. Manchester City ~~
      1     x     2  ovr_und_point  over  under
0  3.1  3.65  2.07            2.5  1.54   2.32

 ~~ MyBookie Odds IMPLIED for Aston Villa v. Manchester City ~~
           1         x         2  point      over     under
0  0.322581  0.273973  0.483092    2.

# Conclusion and Pick
Value pick with the model is Villa, but I'm not ready to wager against the Citizens just yet despite their bad run of form.
### My Pick:

**NO BET**